In [1]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import json

In [2]:
# both ""http://stackoverflow.com/jobs" and "http://careers.stackoverflow.com/jobs" work

In [3]:
r = requests.get("http://stackoverflow.com/jobs",
                 params={"sort": "p", "pg": 2})

In [4]:
r.status_code

200

In [6]:
soup = BeautifulSoup(r.content)

In [7]:
jobs = soup.find_all("div", ["-item", "-job"])

In [8]:
job = jobs[0]

In [10]:
job

<div class="-item -job _highlighted _topspot" data-jobid="82835">
<span class="_badge">
                    featured
                </span>
<span class="fav-toggle" data-href="//careers.stackoverflow.com/users/login?returnUrl=%2Fso-proxy%2Fjobs%3Fsort%3Dp%26pg%3D2" data-jobid="82835"></span>
<div class="-title">
<h1>
<a class="job-link" href="/jobs/82835/java-engineer-with-hadoop-cognitive-medical-systems?a=rMnAVaektIA&amp;offset=4&amp;pg=2&amp;sort=p" title="Java Engineer with Hadoop">Java Engineer with Hadoop</a> </h1>
</div>
<ul class="metadata primary">
<li class="employer">
                    Cognitive Medical Systems
                </li>
<li class="location">
                    San Diego, CA
                </li>
</ul>
<p class="text _muted _small description">Position Summary:
We are looking for a Hadoop System Engineer to design, implement and maintain a large…</p>
<div class="tags">
<p><a class="post-tag job-link no-tag-menu" href="/jobs/tag/hadoop">hadoop</a> <a class="po

In [9]:
job.attrs

{'class': ['-item', '-job', '_highlighted', '_topspot'], 'data-jobid': '82835'}

In [23]:
tags = [each.text for each in job.find_all("a", ["post-tag"])]

In [24]:
tags

['testcomplete', 'javascript', 'java', 'smartbear-testcomplete']

In [27]:
job.find("div", "-title").find("a", "job-link").text

'Test Automation Engineer ATC Global - $15/hr, 100% remote position'

In [30]:
job.find("li", "employer").text.strip()

'Crossover'

In [31]:
job.find("li", "location").text.strip()

'Bengaluru, India'

In [84]:
def process_site(page_n):
    # i need to fake it otherwise I get wrong data
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r = requests.get("http://careers.stackoverflow.com/jobs",
                     params={"sort": "p", "pg": page_n},
                     headers=headers)
    soup = BeautifulSoup(r.content)
    jobs = soup.find_all("div", ["-item", "-job"])
    return [parse_job(job) for job in jobs]

In [85]:
def parse_job(job):
    res = {}
    res["title"] = job.find("div", "-title").find("a", "job-link").text
    res["employer"] = job.find("li", "employer").text.strip()
    res["location"] = job.find("li", "location").text.strip()
    res["featured"] = "_highlighted" in job.attrs["class"]
    res["topspot"] = "_topspot" in job.attrs["class"]
    res["job_id"] = job.attrs["data-jobid"]
    res["tags"] = [each.text for each in job.find_all("a", ["post-tag"])]
    return res

In [86]:
offers_json = []
for page_n in range(1, 41):
    print(page_n, end=" ")
    offers_n = process_site(page_n)
    if len(offers_n) != 25:
        print("Only {} offers".format(len(offers_n)))
    offers_json += offers_n
    sleep(10)

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 Only 4 offers


In [87]:
len(offers_json)

979

In [89]:
json.dump(offers_json, open("careers_so_20160125.json", "w"), indent=2)